In [ ]:
def PreProcessing(dat):
  VerbalConv = []
  with open('VerbalConversion.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
      VerbalConv.append(row)
  QuantConv = []
  with open('QuantConversion.csv') as csv_file:
    csv_reader2 = csv.reader(csv_file, delimiter=',')
    for row in csv_reader2:
      QuantConv.append(row)
  dat=dat[dat[5] == 'Accepted']
  dat=dat[dat[3] == 'MS']
  dat=dat.drop([0, 2, 3, 4, 5, 6, 7, 8, 13, 14, 15, 16, 17, 18], axis=1)
  dat=dat.dropna(axis=0, how='any')
  dat=dat.reset_index(drop=True)
  dat.loc[dat[9] > 4.0, 9]=dat.loc[dat[9] > 4.0, 9]*4/10
  for i, j in VerbalConv:
    dat[10]=dat[10].mask(dat[10] == int(i), int(j))
  for i, j in QuantConv:
    dat[11]=dat[11].mask(dat[11] == int(i), int(j))
  dat[9]=pd.to_numeric(dat[9])
  dat[10]=pd.to_numeric(dat[10])
  dat[11]=pd.to_numeric(dat[11])
  dat[12]=pd.to_numeric(dat[12])
  dat=pd.DataFrame(dat.values, columns=["College Name", "GPA","Verbal","Quants","Writing"])
  dat.to_csv("cleanedDataFinal.csv")
  print('Preprocessing Done')
  return dat


In [ ]:
ProcessedDat = PreProcessing(dat)
ProcessedDat.head()

NameError: ignored

Preprocessing of the data

In [ ]:
def clustering():
  dat = pd.read_csv("cleanedDataFinal.csv")
  dat = dat.drop(['Unnamed: 0'], axis=1)
  UniqueCollegeData = dat.groupby("College Name", as_index=False).mean()
  clusterData = UniqueCollegeData.drop(["College Name"], axis=1)
  no_of_clusters = 5
  model = KMeans(n_clusters=no_of_clusters, init='k-means++', max_iter=100, n_init=1)
  model.fit(clusterData)
  s_score = silhouette_score(clusterData, model.labels_)
  UniqueCollegeDatal['Cluster']= pd.Series(model.labels_)
  print("Clustering Done")
  print('Silhouette Score for Clustering is '+str(s_score))
  return UniqueCollegeData


In [ ]:
clusteredData = clustering()
clusteredData.head()

Clustering and Alloting Tiers

In [ ]:
def ModelCreation(datPoints):
  CollegeTiers = []
  with open('CollegeTiers.csv') as csv_file:
    csv_reader2 = csv.reader(csv_file, delimiter=', ')
    for row in csv_reader2:
      CollegeTiers.append(row[1:])
  datPoints
  datPoints["Tier"] = ""
  for i, j in CollegeTiers:
    datPoints["Tier"] = datPoints["Tier"].mask(datPoints["College Name"] == i, j)
  datPoints = datPoints.drop(['College Name'], axis=1)
  X = np.asarray(datPoints.drop(['Tier'], axis=1).values)
  y= np.asarray (datPoints['Tier'].values)
  X_train, X_test, Y_ train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)
  model= MLPClassifier(alpha=le-4, hidden_layer_sizes=(9, 7), random_state=1, solver='bfgs', max_iter=10000)
  model.fit(X_train, Y_train)
  # model. fit (X, y)
  filename= "CollegeProbPredictor.sav"
  pickle.dump(model,open(filename,'wb'))


In [ ]:
def ModelCreation_predict(datPoints):
  CollegeTiers = []
  with open('CollegeTiers.csv') as csv_file:
    csv_reader2 = csv.reader(csv_file, delimiter=',')
    for row in csv_reader2:
      CollegeTiers.append(row[1:])
  datPoints
  datPoints["Tier"] = ""

  # print (datPoints.shape)
  for i, j in CollegeTiers:
    datPoints["Tier"] = datPoints["Tier"].mask(datPoints["'College Name"] == i, j)
  datPoints = datPoints.drop(['College Name'], axis=1)

  # print (datPoints.shape)
  X= np.asarray(datPoints.drop(['Tier'], axis=1).values)
  y = np.asarray(datPoints['Tier'].values)

  # print (X. shape)
  X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)
  return X_test, Y_test


MLP Model Creation and Training of Dataset

In [ ]:
X_test, Y_test = ModelCreation_predict(ProcessedDat)
MLPmodel = pickle.load(open('CollegeProbPredictor.sav', 'rb'))
Y_pred = MLPmodel.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
CollegeTiers = []
with open('CollegeTiers.csv') as csc_file:
  csv_reader2 = csv.reader(csv_file, delimiter=',')
  for row in csv_reader2:
    CollegeTiers.append(row[1:])

datPoints=pd.read_csv("clusteredDatawithTiers.csv")
datPoints=datPoints.drop(['Cluster', 'Unnamed: 0'], axis=1)
datPoints.head()


Testing of Trained Data (incomplete)

In [ ]:
def predictListCollege(Scores, model):
  Scores = np.asarray(Scores).reshape(1, -1)
  ProbMax = max(max(model.predict_proba(Scores)))
  tier = model.predict(Scores)[0]
  CollegeTierList = {}
  if (tier == '5'):
    nTier = 10
    Neigh1 = NearestNeighbors(n_ neighbors=nTier, p=2, algorithm='auto', metric='euclidean')
    TierUpperPoints = datPoints[datPoints["Tier"] == int(tier)].reset_index(drop=True)
    Neigh1.fit(np.asarray(TierUpperPoints.drop(['College Name', 'Tier'], axis=1).values))
    Indexes = Neighl.kneighbors(Scores, return_distance=False)[0]
    CollegeList = [TierUpperPoints.at[i, 'College Name'] for i in Indexes]
    UTier = []
    for Name in CollegeList:
      NeededVerbal = datPoints[datPoints ["College Name"] == Name]["Verbal"].values[0]
      NeededQuants= datPoints[datPoints ["College Name"] == Name]["Quants"].values[0]
      NeededGPA= datPoints[datPoints ["College Name"] == Name]["GPA"].values[0]
      NeededWriting = datPoints[datPoints ["College Name"] == Name]["Writing"].values[0]
      UTier.append([Name, NeededVerbal, NeededQuants, NeededGPA, NeededWriting])
    CollegeTierList['UpperTier'] = UTier
    CollegeTierList['LowerTier'] = []
  else:
    nTier = int(math.ceil(ProbMax*10))
    nTierLower = int(10-nTier)
    Neigh1 = NearestNeighbors(n_neighbors=nTier, p=2, algorithm= 'auto', metric='euclidean')
    TierUpperPoints = datPoints[datPoints["Tier"] == int(tier)].reset_index(drop=True)
    Neighl.fit(np.asarray(TierUpperPoints.drop(['College Name', 'Tier'], axis=1).values))
    Indexes = Neighl.kneighbors(Scores, return_distance=False) [0]
    CollegeList = [TierUpperPoints.at[i,'College Name'] for i in Indexes]
    UTier = []
    for Name in CollegeList:
      NeededVerbal = datPoints[datPoints["College Name"] == Name]["'Verbal"].values[0]
      NeededQuants = datPoints[datPoints["College Name"] == Name]["Quants"].values[0]
      NeededGPA = datPoints[datPoints["College Name"] == Name] ["GPA"]values[0]
      Neededwriting = datPoints[datPoints["College Name"] == Name]["Writing"].values[0]
      UTier.append([Name, NeededVerbal, NeededQuants, NeededGPA, NeededWriting])
    Neigh2 = NearestNeighbors(n_neighbors=nTierLower, p=2, algorithm='auto', metric=' euclidean')
    TierLowerPoints = datPoints[datPoints["Tier"] == (int(tier)+1)].reset_index(drop = True)
    Neigh2.fit(np.asarray(TierLowerPoints.drop(['College Name', 'Tier'], axis=1).values))
    TierLowerPoints.head()
    Indexs2 = Neigh2.kneighbors(Scores, return_distance=False)[0]
    CollegeList2 = [TierLowerPoints.at[i, 'College Name'] for i in Indexs2]
    LTier = []
    for Name in CollegeList2:
      NeededVerbal = datPoints[datPoints["College Name"] == Name]["Verbal"].values[0]
      NeededQuants = datPoints[datPoints["College Name"] == Name]["Quants"].values[0]
      NeededGPA = datPoints[datPoints["College Name"] == Name]["GPA"].values[0]
      NeededWriting= datPoints[datPoints["College Name"] == Name]["Writing"].values[0]
      LTier.append([Name, NeededVerbal, NeededQuants, NeededGPA, NeededWriting])
    ColleqeTierList['UpperTier'] = UTier


Apply K-NN for calulating nearest neighbors in prediction list

In [ ]:
CollegeList = predictListCollege([3.52,160,166,3.5]),MLPmodel)
#[GPA, VERBAL, QUANTS, WRITING]
for k in CollegeList.keys():
  print(k+":\n\n")
  for college in CollegeList[k]:
    print(college[0]+"\nRequired Verbal Score: "+str(college[1])+" Required Quants Score: "+str(college[2])
    +" Required GPA: "+str(college[3])+" Required Writing Score: "+str(college[4])+"\n")